In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb  2 20:08:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    13W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

##  data set


### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget  -b  https://files.grouplens.org/datasets/movielens/ml-25m.zip

Continuing in background, pid 34694.
Output will be written to ‘wget-log.5’.


In [ ]:
!unzip -o "ml-25m.zip"  -d  "/content"

Archive:  ml-25m.zip
  inflating: /content/ml-25m/tags.csv  
  inflating: /content/ml-25m/links.csv  
  inflating: /content/ml-25m/README.txt  
  inflating: /content/ml-25m/ratings.csv  
  inflating: /content/ml-25m/genome-tags.csv  
  inflating: /content/ml-25m/genome-scores.csv  
  inflating: /content/ml-25m/movies.csv  


In [ ]:
data = pd.read_csv('ml-25m/ratings.csv')#, sep='::', header=0, skipinitialspace=True)
data.dropna(inplace=True)


In [ ]:
data.columns =[ 'user', 'item', 'label','timestamp']

In [ ]:
!pip install pandas openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## graphsage LibRecommender

In [ ]:
!pip install LibRecommender==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip uninstall pycocotools
!pip install  pycocotools

Found existing installation: pycocotools 2.0.0
Uninstalling pycocotools-2.0.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/pycocotools-2.0.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/pycocotools/*
Proceed (Y/n)? y
  Successfully uninstalled pycocotools-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp38-cp38-linux_x86_64.whl size=423025 sha256=247da34146b0c915bb5db67482eb26963a5a0ab32ee6e46f78593b7518f3b414
  Stored in directory: /root/.cache/pip/wheels/3e/08/ac/58126fe59992032701437336493f6132e1b72381a62d00b595
Successfully built pycocotools


In [ ]:
!pip install numpy==1.21.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [ ]:

import tensorflow as tf

from libreco.data import DatasetFeat, DataInfo
from libreco.data import split_by_ratio_chrono,split_by_ratio
from libreco.algorithms import DeepFM
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:

col_user = ["user"]
col_item = ["item"]
col_dense = ["label"]

In [ ]:
train, test = split_by_ratio(data, test_size=0.2)

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(
        train,
       
        shuffle=False,
    )

In [ ]:
test_data = DatasetFeat.build_testset(test, shuffle=False)

In [ ]:
eval_data = DatasetFeat.build_testset(test)
print(data_info)


n_users: 162541, n_items: 37970, data sparsity: 0.3241 %


In [ ]:
import torch
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.algorithms import (
    NGCF,
    LightGCN,
    GraphSage,
    GraphSageDGL,
    PinSage,
    PinSageDGL,
)


    # # only do negative sampling on eval data
    # eval_data.build_negative_samples(
    #     data_info, item_gen_mode="random", num_neg=1, seed=2222
    # )
metrics = [  
        "precision",
        "recall",
        "map",
        "ndcg",
    ]


## Graphsage working with 25M

In [ ]:

graphsage = GraphSage(
        "ranking",
        data_info,
        loss_type="max_margin",
        paradigm="i2i",
        embed_size=16,
        n_epochs=2,
        lr=3e-4,
        lr_decay=False,
        reg=None,
        batch_size=2048,
        num_neg=1,
        dropout_rate=0.0,
        num_layers=1,
        num_neighbors=10,
        num_walks=10,
        sample_walk_len=5,
        margin=1.0,
        sampler="random",
        start_node="random",
        focus_start=False,
        seed=42,
    )
graphsage.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=eval_data,
        metrics=metrics,
    )

# print("batch recommendation: ", graphsage.recommend_user(user=[1, 2, 3], n_rec=7))


Training start time: 2023-02-02 18:49:17


pairwise random walk iterator: 100%|██████████| 10000/10000 [27:15<00:00,  6.11it/s]


Epoch 1 elapsed: 1635.729s
	 train_loss: 0.2048


eval_rec: 100%|██████████| 162519/162519 [03:26<00:00, 785.88it/s]


	 eval precision@10: 0.0230
	 eval recall@10: 0.0166
	 eval map@10: 0.0633
	 eval ndcg@10: 0.0869


pairwise random walk iterator: 100%|██████████| 10000/10000 [26:58<00:00,  6.18it/s]


Epoch 2 elapsed: 1618.085s
	 train_loss: 0.1713


eval_rec: 100%|██████████| 162519/162519 [03:25<00:00, 789.37it/s]


	 eval precision@10: 0.0214
	 eval recall@10: 0.0141
	 eval map@10: 0.0419
	 eval ndcg@10: 0.0641


item embedding: 100%|██████████| 19/19 [00:00<00:00, 25.10it/s]


In [ ]:
print("recommendation: ", graphsage.recommend_user(user=1, n_rec=7))


recommendation:  [(593, 0.9999995), (2959, 0.9999995), (356, 0.9999995), (318, 0.9999995), (858, 0.9999995), (1089, 0.9999994), (527, 0.9999994)]


## Pinsage Not working with 25M

In [ ]:
pinsage = PinSage(
        "ranking",
        data_info,
        loss_type="cross_entropy",
        paradigm="u2i",
        embed_size=16,
        n_epochs=2,
        lr=3e-4,
        lr_decay=False,
        reg=None,
        batch_size=2048,
        num_neg=1,
        dropout_rate=0.0,
        remove_edges=False,
        num_layers=1,
        num_neighbors=10,
        num_walks=10,
        neighbor_walk_len=2,
        sample_walk_len=5,
        termination_prob=0.5,
        margin=1.0,
        sampler="random",
        start_node="random",
        focus_start=False,
        seed=42,
    )
pinsage.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=eval_data,
        metrics=metrics,
    )

## Pinsage Not working with 25M

In [ ]:
train, test = split_by_ratio(data, test_size=0.2)

In [ ]:
train_data, data_info = DatasetPure.build_trainset(
        train,
       
        shuffle=False,
    )

In [ ]:
eval_data = DatasetPure.build_testset(test)
print(data_info)
# only do negative sampling on eval data
# eval_data.build_negative_samples(
#         data_info, item_gen_mode="random", num_neg=1, seed=2222
#     )

n_users: 162541, n_items: 37970, data sparsity: 0.3241 %


In [ ]:
lightgcn = LightGCN(
        "ranking",
        data_info,
        loss_type="bpr",
        embed_size=16,
        n_epochs=2,
        lr=3e-4,
        lr_decay=None,
        reg=0.0,
        batch_size=2048,
        num_neg=1,
        dropout_rate=0.0,
        n_layers=3,
        device=torch.device("cpu"),
    )
lightgcn.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=eval_data,
        metrics=metrics,
    )